<a href="https://colab.research.google.com/github/yenlung/Deep-Learning-Basics/blob/master/colab04d_%E6%89%BE%E5%88%B0%E6%96%87%E5%AD%97%E7%9A%84embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
## 1. 安裝 `transformers` 套件

In [3]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 66.6 MB/s 
     |████████████████████████████████| 6.6 MB 52.8 MB/s 
     |████████████████████████████████| 84 kB 4.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## 2. 使用 Bert 的 Tokenizer

In [4]:
from transformers import BertTokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

一句話來變成 BERT 編碼!

In [6]:
tokenizer(["測試一句話"])

{'input_ids': [[101, 3947, 6275, 671, 1368, 6282, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

我們可以看出這是一個字、一個字切的。

In [ ]:
tokenizer.tokenize("測試一句話")

['測', '試', '一', '句', '話']

真的送進 BERT 應該長這樣。

In [7]:
tokenizer.convert_ids_to_tokens([101, 3947, 6275, 671, 1368, 6282, 102])

['[CLS]', '測', '試', '一', '句', '話', '[SEP]']

In [8]:
tokenizer.convert_ids_to_tokens([3947, 6275, 671, 1368, 6282])

['測', '試', '一', '句', '話']

一次好幾句的試驗!

In [9]:
tokenizer(["測試一句話", "中間夾。", "也許這樣吧。", "買iPhone。"])

{'input_ids': [[101, 3947, 6275, 671, 1368, 6282, 102], [101, 704, 7279, 1933, 511, 102], [101, 738, 6258, 6857, 3564, 1416, 511, 102], [101, 6525, 100, 511, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [10]:
tokenizer(["abc", "David", "蘋果Apple"])

{'input_ids': [[101, 8425, 102], [101, 100, 102], [101, 5981, 3362, 100, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1, 1, 1]]}

In [11]:
tokenizer.convert_ids_to_tokens([8425,8776,8350])

['abc', 'david', 'apple']

## 3. 找到文字的 embedding

In [12]:
from transformers import AutoTokenizer

In [13]:
checkpoint = "bert-base-chinese"

In [14]:
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [15]:
sequence = "我想看看這會變成什麼"

In [19]:
model_inputs = tokenizer2(sequence)

In [22]:
model_inputs["input_ids"]

[101, 2769, 2682, 4692, 4692, 6857, 3298, 6365, 2768, 784, 7938, 102]

In [23]:
from transformers import TFBertModel

In [24]:

model = TFBertModel.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/456M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [27]:
import tensorflow as tf

In [28]:
model_inp = tf.constant([model_inputs["input_ids"]])

In [29]:
model_inp

<tf.Tensor: shape=(1, 12), dtype=int32, numpy=
array([[ 101, 2769, 2682, 4692, 4692, 6857, 3298, 6365, 2768,  784, 7938,
         102]], dtype=int32)>

In [30]:
output = model(model_inp)

In [49]:
h = output['last_hidden_state'].numpy()[0][0]

In [52]:
h

array([-2.99048007e-01,  4.56823051e-01,  5.82943082e-01, -4.11516964e-01,
       -1.71039820e-01, -4.91626292e-01, -6.47809148e-01, -5.26151180e-01,
       -4.55655694e-01,  4.39405113e-01,  3.13840002e-01,  3.97040337e-01,
        2.57390112e-01, -7.44233787e-01,  1.57073975e+00, -6.02495432e-01,
        3.26073140e-01, -1.29350936e+00,  4.72841889e-01, -2.41677999e-01,
       -3.53348821e-01,  3.60392094e-01, -3.66648644e-01, -5.83259344e-01,
        2.78585523e-01,  5.10834813e-01, -1.07827701e-01, -6.50267527e-02,
        4.87270892e-01,  6.90398455e-01,  6.89542368e-02, -2.00402051e-01,
       -7.77645111e-01, -4.56595480e-01, -2.12552428e-01, -5.58262408e-01,
       -3.43980610e-01,  2.74059445e-01,  5.78915477e-02, -4.94238883e-01,
        1.05730891e+00, -9.70214307e-02,  8.59945118e-02,  2.27669859e+00,
        1.93625242e-01, -3.34333181e-01,  3.02552760e-01,  3.14233571e-01,
       -6.46616280e-01,  2.31464982e-01,  3.39613646e-01,  1.01438055e+01,
        2.00304723e+00,  